In [3]:
import tensorflow as tf
print(tf.__version__)


2.1.0


In [7]:
import tensorflow_datasets as tfds
imdb,info=tfds.load("imdb_reviews",with_info=True,as_supervised=True)

In [21]:
import numpy as np

train_data,test_data=imdb['train'],imdb['test']

training_sentences=[]
training_labels=[]
testing_sentences=[]
testing_labels=[]

for s,l in train_data:
  training_sentences.append(s.numpy().decode('utf8'))
  training_labels.append(l.numpy())
  
for s,l in test_data:
  testing_sentences.append(s.numpy().decode('utf8'))
  testing_labels.append(l.numpy())
  
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

type(training_sentences)

list

In [26]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type = 'post'
oov_tok = "<OOV>"

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer=Tokenizer(num_words=vocab_size,oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index=tokenizer.word_index
sequences=tokenizer.texts_to_sequences(training_sentences)
padded=pad_sequences(sequences,maxlen=max_length,truncating=trunc_type)

In [33]:
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)

In [36]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[3]))
print(training_sentences[3])

i am a huge woody allen fan and so when i saw that this was playing at the cinema i couldn't help myself i wanted to see how allen would follow up his magnificent film match point seeing as this is another one of his films shot in g b which is unique among allen's work along with what seems to be his new <OOV> scarlett <OOV> scoop is much lighter than <OOV> and the humor is <OOV> most enjoyable aspect the plot revolves around <OOV> character a <OOV> student who gets a tip on a hot story from beyond the grave she falls in love with a suspected serial killer jackman and she must decide whether the truth is
I am a huge Woody Allen fan and so when I saw that this was playing at the cinema I couldn't help myself. I wanted to see how Allen would follow up his magnificent film Match Point seeing as this is another one of his films shot in G.B. (which is unique among Allen's work) along with what seems to be his new muse Scarlett Johanson. Scoop is much lighter than MP and the humor is Scoop's

In [38]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
flatten_1 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 11526     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [39]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 7s 265us/sample - loss: 0.4965 - accuracy: 0.7406 - val_loss: 0.3458 - val_accuracy: 0.8502
Epoch 2/10
25000/25000 [==============================] - 5s 208us/sample - loss: 0.2389 - accuracy: 0.9092 - val_loss: 0.3576 - val_accuracy: 0.8428
Epoch 3/10
25000/25000 [==============================] - 6s 250us/sample - loss: 0.0905 - accuracy: 0.9770 - val_loss: 0.4520 - val_accuracy: 0.8280
Epoch 4/10
25000/25000 [==============================] - 9s 341us/sample - loss: 0.0216 - accuracy: 0.9974 - val_loss: 0.5293 - val_accuracy: 0.8280
Epoch 5/10
25000/25000 [==============================] - 10s 407us/sample - loss: 0.0054 - accuracy: 0.9996 - val_loss: 0.5969 - val_accuracy: 0.8284
Epoch 6/10
25000/25000 [==============================] - 8s 312us/sample - loss: 0.0017 - accuracy: 1.0000 - val_loss: 0.6482 - val_accuracy: 0.8271
Epoch 7/10
25000/25000 [=========================

In [41]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


In [56]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences([sentence])
padded = pad_sequences(sequences, maxlen=max_length, padding='post', truncating=trunc_type)
print(model.predict(padded))
np.argmax(model.predict(padded))



[[9.9996781e-01]
 [9.9999988e-01]
 [1.0000000e+00]
 ...
 [1.4399976e-05]
 [3.4082227e-04]
 [9.9995399e-01]]


2

In [59]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [60]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')